In [3]:
import os
import json
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import numpy as np
from pprint import pprint
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model
import time

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7729362004872548898
xla_global_id: -1
]


In [16]:
# Load the trained model
model_path = 'big_model.keras'
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file not found: {model_path}")

model = load_model(model_path)
print(f"Model input shape: {model.input_shape}")

# Open the video file
video_path = 'Testing/white_lanes.mp4'
if not os.path.exists(video_path):
    raise FileNotFoundError(f"Video file not found: {video_path}")

video = cv2.VideoCapture(video_path)

if not video.isOpened():
    raise IOError("Error opening video file")

# Get video properties
fps = int(video.get(cv2.CAP_PROP_FPS))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(f"Video properties: {fps} FPS, {width}x{height}")

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, fps, (width, height))

total_frames = 0
start_time = time.time()

while True:
    ret, frame = video.read()
    if not ret:
        print("End of video stream or error reading frame")
        break

    # Convert BGR to RGB if your model expects RGB images
    # If your model was trained on RGB images, uncomment the following line
    # frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Else, use the frame as is
    processed_frame = cv2.resize(frame, (512, 256))  # Resize to match model input

    # Preprocess the frame
    processed_frame = processed_frame / 255.0  # Normalize
    processed_frame = np.expand_dims(processed_frame, axis=0)

    # Predict using the model
    prediction = model.predict(processed_frame, verbose=0)[0]

    # Post-process the prediction
    lane_mask = (prediction > 0.5).astype(np.uint8) * 255  # Binary mask

    # Resize the lane mask to match the original frame size
    lane_mask_resized = cv2.resize(lane_mask, (width, height), interpolation=cv2.INTER_NEAREST)

    # Create a color mask with red color where the lanes are detected
    # Initialize a zero array with the same shape as the original frame
    colored_lanes = np.zeros_like(frame)

    # Set red color where lane_mask is 255
    # In BGR format, red is (0, 0, 255)
    colored_lanes[lane_mask_resized == 255] = [0, 0, 255]

    # Define the kernel for morphological operations
    kernel_size = 5  # Adjust as needed
    kernel = np.ones((kernel_size, kernel_size), np.uint8)

    # Apply morphological closing to connect nearby pixels
    lane_mask_closed = cv2.morphologyEx(lane_mask_resized, cv2.MORPH_CLOSE, kernel)

    # Optionally, apply dilation to thicken the lane lines
    lane_mask_dilated = cv2.dilate(lane_mask_closed, kernel, iterations=1)

    # Create the colored mask with red lanes
    colored_lanes = np.zeros_like(frame)
    colored_lanes[lane_mask_dilated == 255] = [0, 0, 255]

    # Overlay the colored mask onto the original frame
    result = cv2.addWeighted(frame, 0.7, colored_lanes, 0.3, 0)

    # Calculate and display FPS
    total_frames += 1
    elapsed_time = time.time() - start_time
    fps_current = total_frames / elapsed_time
    cv2.putText(result, f"FPS: {fps_current:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                1, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Lane Detection', result)

    # Write the frame to the output video
    out.write(result)

    # Print progress every 100 frames
    if total_frames % 100 == 0:
        print(f"Processed {total_frames} frames. Average FPS: {fps_current:.2f}")

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Control frame rate to match video FPS
    time.sleep(max(1./fps - (time.time() - start_time) % (1./fps), 0))

video.release()
out.release()
cv2.destroyAllWindows()

print(f"Video processing complete. Total frames: {total_frames}")
print(f"Average FPS: {total_frames / elapsed_time:.2f}")

Model input shape: (None, 256, 512, 3)
Video properties: 30 FPS, 1920x1080
Processed 100 frames. Average FPS: 2.61
Processed 200 frames. Average FPS: 2.69
Processed 300 frames. Average FPS: 2.72
Processed 400 frames. Average FPS: 2.73
Processed 500 frames. Average FPS: 2.74
Video processing complete. Total frames: 593
Average FPS: 2.73
